# HH здесь необходимые данные для дальнейшей отладки (подготовки начального состояния)


In [ ]:
from qiskit_nature.units import DistanceUnit
from Ternary_Tree import InitstateTTInfo, ternary_tree
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit import QuantumCircuit
# def initial_state_H2(bond_length):
at_name ="H"
bond_length = 1
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
    basis="sto3g",
#         basis="6-31G",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
# print(es_problem)
nmodes = es_problem.num_spin_orbitals
num_alpha = es_problem.num_alpha
num_beta = es_problem.num_alpha
if num_alpha != num_beta:
    print("Все плохо")

# tt = ternary_tree(nmodes, num_particles = num_alpha*2)
print(tt)
num_list = None
# num_list = []
# for i in range(nmodes):
#     num_list.append([2*i,2*i + 1])

# Itti = InitstateTTInfo(nmodes,num_list)
# # qc = QuantumCircuit
# #     return 
# print(Itti.oqe)
# print(Itti.tqe)
# print(Itti.pt)
qc = QuantumCircuit(4)
qc.x(0)
qc.x(3)
qc.x(1)
qc = qc.compose(PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('IXXI'), pi/2)).to_circuit())
qc = qc.compose(PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('XZZZ'), pi/2)).to_circuit())
qc = qc.compose(PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('XZIZ'), pi/2)).to_circuit())
qc = qc.compose(PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('YIIZ'), pi/2)).to_circuit())



print(qc)
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCC,UCCSD
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter, BravyiKitaevMapper
from Ternary_Tree import TernaryTreeMapper
# mapper = TernaryTreeMapper(es_problem)
mapper = BravyiKitaevMapper()
mapper = JordanWignerMapper()
converter = QubitConverter(mapper)
# : tuple[int, int] | None = None,
        
u = UCC(qubit_converter = converter,  num_spatial_orbitals = 2, num_particles = (1,1) ,excitations = 'sd')
print(u.decompose().decompose().decompose().depth())
print(u.decompose().decompose().decompose())

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
import Ternary_Tree as tt
import numpy as np
from qiskit import QuantumCircuit
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC
from qiskit.algorithms.optimizers import L_BFGS_B, SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCC,UCCSD
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter, BravyiKitaevMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.quantum_info import Statevector
from qiskit.primitives.utils import (
    _circuit_key,
    _observable_key,
    bound_circuit_to_instruction,
    init_circuit,
    init_observable,
)
from qiskit.quantum_info.operators import Pauli
from qiskit.opflow import PauliTrotterEvolution, PauliOp
from numpy import pi

numpy_solver = NumPyMinimumEigensolver()

at_name ="H"
vqe_solver = VQEUCCFactory(Estimator(), UCC(excitations = 'sd'),  SLSQP())
def jwenergy(bond_length):
    driver = PySCFDriver(
        atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
        basis="sto3g",
#         basis="6-31G",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    es_problem = driver.run()
    mapper = JordanWignerMapper()
    converter = QubitConverter(mapper)
    main_operator = converter.convert(
            es_problem.second_q_ops()[0],
            num_particles=es_problem.num_particles,
            sector_locator=es_problem.symmetry_sector_locator,
        )
    
    vqe_solver = VQEUCCFactory(Estimator(), UCC(excitations = 'sd'),  SLSQP())

    calc = GroundStateEigensolver(converter, vqe_solver)
    res = calc.solve(es_problem)
#     inenergy = final_state.expectation_value(main_operator)
    return res

def ttenergy(bond_length):
    
    driver = PySCFDriver(
        atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
        basis="sto3g",
#         basis="6-31G",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    es_problem = driver.run()
    mapper = tt.TernaryTreeMapper(es_problem)
    converter = QubitConverter(mapper)
    main_operator = converter.convert(
            es_problem.second_q_ops()[0],
            num_particles=es_problem.num_particles,
            sector_locator=es_problem.symmetry_sector_locator,
        )
    
    
    qc = QuantumCircuit(4)
    qc.x(0)
#     qc.x(2)
#     qc.x(3)
    qc.x(1)
    
#     qc.x(0)
#     qc.x(2)
    qc.x(3)
    qc.x(1)
    qc = qc.compose(PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('IIXI'), pi/4)).to_circuit())
    qc = qc.compose(PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('IXXI'), pi/4)).to_circuit())
    qc = qc.compose(PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('XZZZ'), pi/4)).to_circuit())
    qc = qc.compose(PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('XZIZ'), pi/4)).to_circuit())
    qc = qc.compose(PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('YIIZ'), pi/4)).to_circuit())
    final_state = Statevector(qc)
    
    vqe_solver = VQEUCCFactory(Estimator(), UCC(excitations = 'sd'),  SLSQP(), initial_state = qc)
    calc = GroundStateEigensolver(converter, vqe_solver)
    res = calc.solve(es_problem)
    inenergy = final_state.expectation_value(main_operator)
    return res, inenergy




N = 10
GE = [0]*N
IE = [0]*N
JWE = [0]*N
R = np.linspace(0.4, 2, N)
for n in range(N):
    print(n)
    GE[n],IE[n] = ttenergy(R[n])
    JWE[n]= jwenergy(R[n])
    IE[n] = IE[n] + GE[n].nuclear_repulsion_energy
    GE[n] = GE[n].nuclear_repulsion_energy + GE[n].groundenergy
    JWE[n] = JWE[n].nuclear_repulsion_energy + JWE[n].groundenergy

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


fig = plt.figure()
ax = fig.add_subplot(1,1,1)
k= 0
f = 100
ax.plot(R[k:f],GE[k:f],label="TernaryTree in UCCSD")
ax.set_xlabel('bond_length')
ax.set_ylabel('energy')
ax.plot(R[k:f],IE[k:f], label="Hartree Fock (quantum initial energy)")
ax.plot(R[k:f],JWE[k:f], label="JW optimization")
# ax.plot(R[k:f],E_HF[k:f], label="Hartree Fock (classic initial energy)")
# ax.plot(R[k:f],E_CISD[k:f], label="Classic CISD")
ax.legend()

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
import Ternary_Tree as tt
import numpy as np
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC
from qiskit.algorithms.optimizers import L_BFGS_B, SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCC,UCCSD
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter, BravyiKitaevMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.quantum_info import Statevector
from qiskit.primitives.utils import (
    _circuit_key,
    _observable_key,
    bound_circuit_to_instruction,
    init_circuit,
    init_observable,
)
numpy_solver = NumPyMinimumEigensolver()

at_name ="H"
vqe_solver = VQEUCCFactory(Estimator(), UCC(excitations = 'sd'),  SLSQP())


def energy(bond_length):
    
    driver = PySCFDriver(
        atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
        basis="sto3g",
#         basis="6-31G",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    es_problem = driver.run()

    
#     converter = QubitConverter(JordanWignerMapper())
#     initial_state = HartreeFock(es_problem.num_spatial_orbitals, es_problem.num_particles, converter)
    
    
    mapper = tt.TernaryTreeMapper(es_problem)
    converter = QubitConverter(mapper)
    main_operator = converter.convert(
            es_problem.second_q_ops()[0],
            num_particles=es_problem.num_particles,
            sector_locator=es_problem.symmetry_sector_locator,
        )
    initial_state = tt.TT_initial_state()

    final_state = Statevector(bound_circuit_to_instruction(initial_state))
#     print(initial_state)
    
#     vqe_solver = VQEUCCFactory(Estimator(), UCC(excitations = 's'),  SLSQP(), initial_state = initial_state)
    calc = GroundStateEigensolver(converter, vqe_solver)
#     calc = GroundStateEigensolver(converter, numpy_solver)
    
    
#     print(main_operator)
   
#     converter2 = QubitConverter(BravyiKitaevMapper())
    converter2 = QubitConverter(JordanWignerMapper())
    main_operator2 = converter2.convert(
        es_problem.second_q_ops()[0],
        num_particles=es_problem.num_particles,
        sector_locator=es_problem.symmetry_sector_locator,
    )
    initial_state2 = HartreeFock(es_problem.num_spatial_orbitals, es_problem.num_particles, converter2)
    final_state2 = Statevector(bound_circuit_to_instruction(initial_state2))
#     res = calc.solve(es_problem)
    
    print(type(es_problem.second_q_ops()[0]))
    print(main_operator)
    print(main_operator2)
    print(initial_state)
    print(initial_state2)
    print(es_problem.second_q_ops()[0])
    k=3
#     return res, inenergy
    return final_state.expectation_value(main_operator[:k]), final_state2.expectation_value(main_operator2[:k])

#     return inenergy + es_problem.nuclear_repulsion_energy, inenergy + es_problem.nuclear_repulsion_energy


# def energy(bond_length):
    
#     driver = PySCFDriver(
#         atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
# #         basis="6-31G",
#         charge=0,
#         spin=0,
#         unit=DistanceUnit.ANGSTROM,
#     )
#     es_problem = driver.run()

    
# #     converter = QubitConverter(JordanWignerMapper())
# #     initial_state = HartreeFock(es_problem.num_spatial_orbitals, es_problem.num_particles, converter)
    
    
#     mapper = tt.TernaryTreeMapper(es_problem)
#     converter = QubitConverter(mapper)
#     main_operator = converter.convert(
#             es_problem.second_q_ops()[0],
#             num_particles=es_problem.num_particles,
#             sector_locator=es_problem.symmetry_sector_locator,
#         )
    
#     initial_state = tt.TT_initial_state(mapper)

    
#     vqe_solver = VQEUCCFactory(Estimator(), UCC(excitations = 's'),  SLSQP(), initial_state = initial_state)
#     calc = GroundStateEigensolver(converter, vqe_solver)
# #     calc = GroundStateEigensolver(converter, numpy_solver)
    
    

#     res = calc.solve(es_problem)
#     final_state = Statevector(bound_circuit_to_instruction(initial_state))
#     inenergy = final_state.expectation_value(main_operator)
#     k=2
#     return res, inenergy


N = 10
GE = [0]*N
IE = [0]*N

R = np.linspace(0.4, 2, N)
for n in range(N):
    print(n)
    GE[n],IE[n] = energy(R[n])
    IE[n] = IE[n] + GE[n].nuclear_repulsion_energy
    GE[n] = GE[n].nuclear_repulsion_energy + GE[n].groundenergy


In [ ]:
import pyscf
def energy_classic(bond_length):

    mol = pyscf.M(
        atom = "H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
        basis = 'sto3g')

    mf = mol.HF().run()
    mycc = mf.CISD().run()
#     print('RCISD correlation energy', mycc.e_corr)

#     mf = mol.UHF().run()
#     mycc = mf.CISD().run()
#     print('UCISD correlation energy', mycc.e_corr)
    return mycc.e_tot, mf.e_tot

mycc = energy_classic(1)

E_CISD = [0]*N
E_HF = [0]*N
for n in range(N):
    print(n)
    E_CISD[n], E_HF[n] = energy_classic(R[n])


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


fig = plt.figure()
ax = fig.add_subplot(1,1,1)
k= 0
f = 100
ax.plot(R[k:f],GE[k:f],label="TernaryTree in UCCSD")
ax.set_xlabel('bond_length')
ax.set_ylabel('energy')
ax.plot(R[k:f],IE[k:f], label="Hartree Fock (quantum initial energy)")
# ax.plot(R[k:f],E_HF[k:f], label="Hartree Fock (classic initial energy)")
# ax.plot(R[k:f],E_CISD[k:f], label="Classic CISD")
ax.legend()

In [ ]:
tt = ternary_tree(12)
tt.build_alpha_beta_tree()
# print( tt.branches())
print(tt)

In [ ]:
from Ternary_Tree import ternary_tree, pauli_table_TT
from Ternary_Tree.pauli_tables import _pauli_table_TT
import copy

import re
dict_prod = {"II" : "I", "XX" : "I", "YY" : "I","ZZ" : "I", 
             "XY" : 'Z',"YX" : 'Z',"XZ" : 'Y',"ZX" : 'Y',"YZ" : 'X',"ZY" : 'X',
            "IX" : "X","XI" : "X","YI" : "Y","IY" : "Y","IZ" : "Z","ZI" : "Z"}

def count_non_I(syms):
    d = {'I': 0,'X': 1,'Y': 1,'Z': 1}
    n = 0
    for sym in syms:
        n += d[sym]
    return n
    
class InitstateTTInfo:
    def __init__(self,nmodes,num_list = None):
        self.nmodes = nmodes 
        self.num_list = num_list
        self.oqe, self.tqe = self.get_enenglement() #[tuple(1,"X")] , [tuple([n,m], ["IX","IY"...])]
        self.prod = self.prod_table(_pauli_table_TT(nmodes))
    def prod_table(self, pauli_table_str):
        def prod_pauli(g1,g2):
            g1 = re.findall("[XYZI]",g1)
            g2 = re.findall("[XYZI]",g2)
            prod = [None]*len(g1)
            for i in range(len(g1)):
                prod[i] = dict_prod[g1[i] + g2[i]]
            return prod
        prod = [None]*len(pauli_table_str)
        for i, a in enumerate(pauli_table_str):
            prod[i] = ''.join(prod_pauli(pauli_table_str[i][0],pauli_table_str[i][1]) ) 

        for i, a in enumerate(prod):
            prod[i] = prod[i]
        return prod


    
    def get_enenglement(self):
        prod = self.prod_table(_pauli_table_TT(self.nmodes, self.num_list))
        qubit_list = [i for i in range(self.nmodes)]
        oqe = self._one_qubit_entenglement(prod, qubit_list)
        tqe = self._two_qubits_entanglement(qubit_list, prod)
        return oqe, tqe
        
    def _one_qubit_entenglement(self, prod, qubit_list):
        n_qubits = len(qubit_list)
        L = len(prod)
        sym_qub = []
        for n in qubit_list:
            sym = ''
            for j in range(L):
                if prod[j][n] not in sym:
                    sym += prod[j][n]
            if len(sym) < 3:
                sym_qub.append(tuple([n ,re.findall("[^I]",sym)]))
        for n in sym_qub:
            qubit_list.remove(n[0])
        return  sym_qub
    
    def _two_qubits_entanglement(self, qubit_list, prod):
        possible_pairs = copy.deepcopy( self._possible_pairs(prod,qubit_list) )
        tqe = []
        L = 0
        def _get_dict():
            diff_number_pair = {}
            nonlocal L 
            L = len(possible_pairs)
            
            for j in range(L):
                pair = possible_pairs[j]
                diff_number_pair[pair[0]] = diff_number_pair.get(pair[0],[]) + [pair]
                diff_number_pair[pair[1]] = diff_number_pair.get(pair[1],[]) + [pair]
            return diff_number_pair
        
        diff_number_pair = _get_dict()
        while len(diff_number_pair):
            min1, key1, min2, key2 = [self.nmodes]*4

            for key in diff_number_pair:
                l = len(diff_number_pair[key])
                if l < min1:
                    min1 = l
                    key1 = key
            for _pair in diff_number_pair[key1]:
                i = _pair.index(key1)
                l = len(diff_number_pair[_pair[i - 1]])
                if l <min2:
                    min2 = l
                    key2 = _pair[i - 1]
                    pair = _pair
            tqe.append(tuple([pair, self.get_str_pairs(prod,pair)]))
            for i in range(L-1,-1,-1):
                if pair[0] in possible_pairs[i] or pair[1] in possible_pairs[i]:
                    possible_pairs.pop(i)
            diff_number_pair = _get_dict()
        return tqe
    
    def _possible_pairs(self, prod, qubit_list):
        possible_pair = []
#         print(prod)
        L = len(prod)
#         print(qubit_list)
        for i, n in enumerate(qubit_list):
            for k in qubit_list[i + 1:]:            
                for j in range(0,L):
                    l = count_non_I(prod[j][n] + prod[j][k])
                    if l % 2 != 0:
                        break
                        
                if l % 2 == 0:
                    possible_pair.append([n,k])
#         print(possible_pair)
        return possible_pair
    
    def get_str_pairs(self,prod,pair):
        str_pair = []
        L = len(prod)
        sym_qubs = []
        for j in range(L):
            sym_qubs.append(prod[j][pair[0]] + prod[j][pair[1]])
        return sym_qubs
    


for n in range(4,30,2):
#     _pauli_table_TT(nmodes)
# n = 10
    num_list = None
    num_list = []
    for i in range(n):
        num_list.append([2*i,2*i + 1])
    Itti = InitstateTTInfo(n,num_list)
# for s in Itti.prod:
#     print(s)
# print(Itti.tqe)
# print(Itti.oqe)
    
    tt = ternary_tree(n)
    tt.build_alpha_beta_tree()
#     print( tt.branches())
#     print(tt)
    print(len(Itti.tqe)*2 + len(Itti.oqe),":",n)
# print(check_two_entanglement(prod))
# print(re.findall("[XYZI]",pttt[0][0]))

In [ ]:
print(_pauli_table_TT(10))

In [ ]:
n_qubits = 5
ng = 8
num_alpha = 1
num_beta = 1
info = tree_generator(n_qubits, ng = ng, num_alpha = num_alpha, num_beta = num_beta,flag = True)
# print(info[1])
inf_info(info)

In [ ]:
data = [[5,8,1,1],[4,8,1,1],[6,8,1,1],[8,8,1,1],[6,12,2,2],[8,12,2,2]]
Y = []
for dat in data:
    n_qubits = dat[0]
    ng = dat[1]
    num_alpha = dat[2]
    num_beta = dat[3]
    info = tree_generator(n_qubits, ng = ng, num_alpha = num_alpha, num_beta = num_beta,flag = True)
    Y += [info[0]]

In [1]:
from Ternary_Tree import ternary_tree
from Ternary_Tree.ternary_tree import EnumInfo, NodeInfo
import copy
gate_name = {0: 'X', 1: 'Y', 2: 'Z'}
gate_index = {'X': 0,'Y': 1, 'Z': 2}
dict_prod = {"II" : "I", "XX" : "I", "YY" : "I","ZZ" : "I", 
             "XY" : 'Z',"YX" : 'Z',"XZ" : 'Y',"ZX" : 'Y',"YZ" : 'X',"ZY" : 'X',
            "IX" : "X","XI" : "X","YI" : "Y","IY" : "Y","IZ" : "Z","ZI" : "Z"}
class test_ternary_tree(ternary_tree):
#     ищет первую занятую x или y ветвь
    def tojw(self):
        s = []
        def _find_first_branch(node):
            print("first_node = ", node)
            if self.parent_child[node][0]:
                return [node,0]
            if self.parent_child[node][1]:
                return [node,1]
            if self.parent_child[node][2]:
                return _find_first_branch(self.parent_child[node][2])
            return [False, False]
# ищет первую не занятую z вершину   
        def _find_second_branch(node,edge):
            parents,flag = -1, False
            def down(parent):
                nonlocal parents, flag
                for index, child in enumerate(self.parent_child[parent].childs):
                    if flag:
                        break
                    if parent == node and index == edge:
                        i=0
                    elif isinstance(child,EnumInfo) and index == 2:                        
                        parents = parent
                        flag = True
                    elif child:
                        down(child)
            down(0)
            return parents
        def _check_xy():
            parentf = 0 
            indexf = 0
            def down(parent):
                nonlocal parentf,indexf 
                for index, child in enumerate(self.parent_child[parent].childs):
                    if isinstance(child, bool):
                        parentf, indexf = parent, index 
                    if child:
                        down(child)
            down(0)
            return [parentf,indexf]
        def check_pair(pair):
            if min(pair[0],pair[1]) % 2 == 1 and abs(pair[0] - pair[1]) == 1:
                return True
            else:
                return False
        def find_trans(pair_list,index):
            pair = pair_list[index]
            indexes = []
            if check_pair(pair):
                return False
            for i, ipair in enumerate(pair_list[index + 1:]):
                if check_pair([ipair[0],pair[0]]):
                    return [2*index + 1, 2*(i + index + 1) ]
                if check_pair([ipair[0],pair[1]]):
                    return [2*index, 2*(i + index + 1) ]
                if check_pair([ipair[1],pair[0]]):
                    return [2*index + 1, 2*(i + index + 1) + 1]
                if check_pair([ipair[1],pair[1]]):
                    return [2*index, 2*(i + index + 1) + 1]
                
        first, index = _check_xy()
#       Здесь сводится все к дереву JW
        s.append(self.branch_transposition(first, index, first, 2))
        while True:
            fl = _find_first_branch(0)
            if fl[0] or fl[0] is 0:
                s.append(self.branch_transposition(fl[0], fl[1], _find_second_branch(fl[0],fl[1]), 2))
            else:
                break
        print(self)
#       Далее просто переставляются оставшиеся ветви
        branches, num_list = self.branches(True)
        num_list = [i.num for i in num_list]
        pair_list = [[num_list[2*i], num_list[2*i+1]] for i in range(len(num_list)//2)]
        L = len(pair_list)
        for index in range(L):
            k = find_trans(pair_list, index)
            if k:
                print('index = ', index)
                print('k = ', k)
                q1 = branches[k[0]][-1]
                q2 = branches[k[1]][-1]
                s.append(self.branch_transposition(q1[0], gate_index[q1[1]], q2[0], gate_index[q2[1]]))
            print(self)
            branches, num_list = self.branches(True)
            num_list = [i.num for i in num_list]
            pair_list = [[num_list[2*i], num_list[2*i+1]] for i in range(len(num_list)//2)]
        
        return s   
    
    
tt = test_ternary_tree(6)
print(tt)
print(tt.parent_child)
print(tt.branches())
s = tt.tojw()
print(tt.branches(True)[1])
num_list = [i.num for i in tt.branches(True)[1]]
pair_list = [[num_list[2*i], num_list[2*i+1]] for i in range(len(num_list)//2)]


def check_pair(pair):
    if min(pair[0],pair[1]) % 2 == 1 and abs(pair[0] - pair[1]) == 1:
        return True
    else:
        return False
def find_trans(pair_list,index):
    pair = pair_list[index]
    indexes = []
    for i, ipair in enumerate(pair_list):
        if (check_pair([ipair[0],pair[0]]) or check_pair([ipair[0],pair[1]]) or
        check_pair([ipair[1],pair[0]]) or check_pair([ipair[1],pair[1]])):
            indexes.append(i)
    return indexes       
print(num_list)
print(pair_list)
print(find_trans(pair_list, 0))
print(s)
                                                          

<>:76: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:76: SyntaxWarning: "is" with a literal. Did you mean "=="?


ImportError: cannot import name 'ternary_tree' from 'Ternary_Tree' (/home/fisher/MyProjects/Ternary_Tree/__init__.py)

In [ ]:
nmodes = 4
num_list = []
for i in range(nmodes//2):
    num_list += [i,nmodes - i - 1]
for i in range(nmodes + nmodes - 1, nmodes + nmodes//2 - 1, -1):
    num_list += [i,-i + nmodes*3 - 1]
print(num_list)
tt = test_ternary_tree(nmodes,enum_list = num_list)
op = 
print(tt.parent_child)
print(tt)
print(tt.copy_node(1))
print(tt.closest_parent(1,2))
print(tt.branch_transposition(0,0, 3, 2))

In [ ]:
num = 123
len(num)

In [ ]:
from Ternary_Tree import InitstateTTInfo
nmodes = 2
itt = InitstateTTInfo(nmodes)
pauli = 'IXII'
dict_prod = {"II" : ["I",1] , "XX" :  ["I",1] , "YY" :  ["I",1] ,"ZZ" :  ["I",1] , 
             "XY" :  ["Z",1j] ,"YX" : ["Z", - 1j],"XZ" : ["Y",-1j], "ZX" : ["Y",1j],"YZ" : ["X",1j],"ZY" : ["X",-1j],
            "IX" : ["X",1], "XI" : ["X",1], "YI" : ["Y",1],"IY" : ["Y",1],"IZ" : ["Z",1],"ZI" : ["Z",1]}

def prodexp(pt, pauli,signphi = False):
    prodpt = []
    coef = []
    coefpt = pt[1]
    pt = pt[0]
    def prod(paulstr1,paulstr2,coef1 = 1):
        newpaulstr = ""
        if not signphi:
            coefpr1 = coef1*1j
        else:
            coefpr1 = -coef1*1j
        coefpr2 = coefpr1
        
        for i in range(len(paulstr1)):
            newpaulstr += dict_prod[paulstr1[i] + paulstr2[i]][0] 
            coefpr1 = coefpr1*(dict_prod[paulstr1[i] + paulstr2[i]][1])
            coefpr2 =  coefpr2*(dict_prod[paulstr2[i] + paulstr1[i]][1])
#             print(newpaulstr, coefpr1, coefpr2)
        if coefpr2 == coefpr1:
            return paulstr1,coef1
        else:
            return newpaulstr, coefpr1
    for i in range(len(pt)):
        newpaulstr1, coefpr1 = prod(pt[i][0],pauli,coefpt[i][0])
        newpaulstr2, coefpr2 = prod(pt[i][1],pauli,coefpt[i][1])
        prodpt.append([newpaulstr1,newpaulstr2])
        coef.append([coefpr1,coefpr2])
    return [prodpt,coef]

print(itt.pttt[0])
print(itt.pttt[1])
print(itt.ptjw)
s = "YI"
print(s)
itt.pttt = prodexp(itt.pttt, s, signphi = False)
# itt.pttt = prodexp(itt.pttt, 'IXXI',signphi = False)
print(itt.pttt[0])
print(itt.pttt[1])# print[[1,-1j]]*3)
s = 'ZZ'
print(s)
itt.pttt = prodexp(itt.pttt, s, signphi = False)

print(itt.pttt[0])
print(itt.pttt[1])


In [ ]:
nmodes = 
itt = InitstateTTInfo(nmodes)
print(itt.pttt[0])
print(itt.pttt[1])
s = "YIIZ"
print(s)
itt.pttt = prodexp(itt.pttt, s, signphi = False)
# itt.pttt = prodexp(itt.pttt, 'IXXI',signphi = False)
print(itt.pttt[0])
print(itt.pttt[1])# print[[1,-1j]]*3)

s = "XZIZ"
print(s)
itt.pttt = prodexp(itt.pttt, s, signphi = False)
# itt.pttt = prodexp(itt.pttt, 'IXXI',signphi = False)
print(itt.pttt[0])
print(itt.pttt[1])# print[[1,-1j]]*3)
s = "XZZZ"
print(s)
itt.pttt = prodexp(itt.pttt, s, signphi = False)
# itt.pttt = prodexp(itt.pttt, 'IXXI',signphi = False)
print(itt.pttt[0])
print(itt.pttt[1])# print[[1,-1j]]*3)
s = "IXXI"
print(s) 
itt.pttt = prodexp(itt.pttt, s, signphi = False)
# itt.pttt = prodexp(itt.pttt, 'IXXI',signphi = False)
print(itt.pttt[0])
print(itt.pttt[1])# print[[1,-1j]]*3)
s = "IIXI"
print(s) 
itt.pttt = prodexp(itt.pttt, s, signphi = False)
# itt.pttt = prodexp(itt.pttt, 'IXXI',signphi = False)
print(itt.pttt[0])
print(itt.pttt[1])# print[[1,-1j]]*3)
from qiskit.quantum_info.operators import Pauli
from qiskit.opflow import PauliTrotterEvolution, PauliOp
from numpy import pi
# Через какие то костыли буду это делать
qc = PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('YIIZ'), pi/4))
qc += PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('XZIZ'), pi/4))
qc += PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('XZZZ'), pi/4))
qc += PauliTrotterEvolution().evolution_for_pauli(PauliOp(Pauli('IXXI'), pi/4))
print(qc)

# Здесь будет подсчет среднего веса для джордана вигнера. 
Для этого необходимо понять как взаимодействовать с puali_table в qiskit

In [ ]:

# from qiskit_nature.mappers.second_quantization.jordan_wigner_mapper import 
import numpy as np
from qiskit.quantum_info.operators import Pauli
from Ternary_Tree import getinf
N = 30
YJWO = []
YBKO = []
YTTO = []
YJW = []
YBK = []
YTT = []
YTT1 = []
Y_JW = []
Y_BK = []
Y_TT = []
Y_TT1 = []
file = open("jwobkotto.txt", "r")
K = int(file.readline())
YJWO = file.readline()
YBKO = file.readline()
# file.read(str(YTTO) + "\n")
file.close()

import re
def strtolist(Y):
    H = re.findall('\d+.\d+',Y)
    Y = [0]*(len(H)//3)
    for i in range(len(H)//3):
        Y[i] = [float(H[3*i]),float(H[3*i + 1]),float(H[3*i + 2])]
    return Y
# YJWO = strtolist(YJWO)
# YBKO = strtolist(YBKO)

        
for nmodes in range(4,N,2):
    num_list = None
#     num_list = []
#     for i in range(nmodes):
#         num_list.append([2*i,2*i + 1])
    num_alpha = nmodes // 4
    num_beta = nmodes // 4
    print("1")
#     YJWO.append(getinf(nmodes, num_alpha, num_beta, mapper_name = "JW", flag_transposition = True))
#     YBKO.append(getinf(nmodes, num_alpha, num_beta, mapper_name = "BK", flag_transposition = True))
#     YTTO.append(getinf(nmodes, num_alpha, num_beta, mapper_name = "TT", flag_transposition = True))

    YJW.append(getinf(nmodes, num_alpha, num_beta, mapper_name = "JW", flag_transposition = False))
    YBK.append(getinf(nmodes, num_alpha, num_beta, mapper_name = "BK", flag_transposition = False))
    YTT.append(getinf(nmodes, num_alpha, num_beta, mapper_name = "TT", flag_transposition = False, num_list = num_list))
    


In [1]:
from Ternary_Tree import ternary_tree,InitstateTTInfo, prod_exp, TernaryTreeMapper
class 
nmodes = 10
num_list = []
for i in range(nmodes//2):
    num_list += [i,nmodes - i - 1]
for i in range(nmodes + nmodes - 1, nmodes + nmodes//2 - 1, -1):
    num_list += [i,-i + nmodes*3 - 1]
tt = ternary_tree(nmodes, enum_list = num_list)
print(tt.branches())
# print(tt)
s = []
print(tt)
s.append(tt.branch_transposition(0,0, 9, 2))
print(tt)
print(s[-1])
s.append(tt.branch_transposition(3,0, 6, 2))
print(tt)
print(s[-1])

s.append(tt.branch_transposition(0,1, 8, 2))
print(tt)
print(s[-1])

s.append(tt.branch_transposition(3,1, 4, 2))
print(tt)
print(tt.height)

print(s[-1])
s.append(tt.branch_transposition(1,0, 5, 2))
print(tt)
print(tt.height)
print(s[-1])
s.append(tt.branch_transposition(1,1, 7, 2))
print(tt)
print(s[-1])
s.append(tt.branch_transposition(2,1, 2, 2))
print(tt)
print(s[-1])
s.append(tt.branch_transposition(0,0, 1, 1))
print(tt)
print(s[-1])
s.append(tt.branch_transposition(9,0, 7, 0))
print(tt)
print(s[-1])

s.append(tt.branch_transposition(3,0, 6, 1))
print(tt)
print(s[-1])
s.append(tt.branch_transposition(1,1, 5, 1))
print(tt)
print(s[-1])
s.append(tt.branch_transposition(6,1, 5, 0))
print(tt)
print(s[-1])
s.append(tt.branch_transposition(5,1, 4, 1))
print(tt)
print(s[-1])

s.append(tt.branch_transposition(4,1, 2, 0))
print(tt)
print(s[-1])
# s.append(tt.branch_transposition(0,1, 3, 1))
# print(tt)
# print(s[-1])
# print(s)

itt = InitstateTTInfo(nmodes)

for sym in s:
    itt.pttt = prod_exp(itt.pttt, sym, signphi = False)
    
print(itt.pttt[0])

print(itt.pttt[1])
print(itt.check_signs())

SyntaxError: invalid syntax (428092637.py, line 2)

In [2]:
if isinstance(0,bool):
    print("asdf")
    

In [34]:
False is 0

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_136057/249076228.py:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  False is 0


False

In [35]:
False == 0

True

In [1]:
from Ternary_Tree import TT_initial_state
print(TT_initial_state(6,[1,1]) )

[[0, 5], [1, 4], [2, 3], [11, 6], [10, 7], [9, 8]]
first_node =  0
first_node =  0
first_node =  0
first_node =  3
first_node =  5
first_node =  1
first_node =  0
first_node =  3
first_node =  5
first_node =  1
first_node =  4
first_node =  2
0X  0Y  0Z  0Z  0Z  0Z  0Z  0Z  0Z  0Z  0Z  0Z  
        3X  3Y  3Z  3Z  3Z  3Z  3Z  3Z  3Z  3Z  
                5X  5Y  5Z  5Z  5Z  5Z  5Z  5Z  
                        1X  1Y  1Z  1Z  1Z  1Z  
                                4X  4Y  4Z  4Z  
                                        2X  2Y  
7   5   10  12  11  9   4   6   1   3   2   8   
 ---------------------------------------------------------------
index =  0
k =  [0, 7]
0X  0Y  0Z  0Z  0Z  0Z  0Z  0Z  0Z  0Z  0Z  0Z  
        3X  3Y  3Z  3Z  3Z  3Z  3Z  3Z  3Z  3Z  
                5X  5Y  5Z  5Z  5Z  5Z  5Z  5Z  
                        1X  1Y  1Z  1Z  1Z  1Z  
                                4X  4Y  4Z  4Z  
                                        2X  2Y  
6   5   10  12  11  9   4   7   

In [2]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
import itertools
at_name = 'Li'
bond_length = 1
basis = 'sto-3g'
nmodes = 4
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
    basis=basis,
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
transformer = ActiveSpaceTransformer(num_electrons = es_problem.num_alpha + es_problem.num_beta - 2, 
                                     num_spatial_orbitals = nmodes//2  , active_orbitals = [1,2])
es_problem = transformer.transform(es_problem)
import numpy as np
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.synthesis import LieTrotter
from Ternary_Tree import energy_classic, TernaryTreeMapper,TT_initial_state, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC
from qiskit import QuantumCircuit
pass_manager = generate_preset_pass_manager(optimization_level = 3)

# for trans in itertools.permutations([0,1,2,3,4,5], r=None):
# for trans in [(1, 0, 2, 3)]: #jw
qc = 0 
def gate_number(name = 'TT', trans = [0,1,2,3],ret = False):
    global u
    k = 10000
#     for trans in [(1,0, 2, 3)]: #jw
    for trans in [(3,2, 1, 0)]: #jw
#     for trans in itertools.permutations([0, 1, 2, 3],r = None):
        if name == "TT":
            mapper = TernaryTreeMapper(es_problem,trans)
            converter = QubitConverter(mapper)
            qc = TT_initial_state(es_problem.num_spin_orbitals, es_problem.num_particles)
        elif name == 'JW':
            
            mapper = JordanWignerMapper(trans)
            converter = QubitConverter(mapper)
            qc = HartreeFock(es_problem.num_spatial_orbitals, es_problem.num_particles, converter)
        else:
            mapper = BravyiKitaevMapper(trans)
            converter = QubitConverter(mapper)
            qc = HartreeFock(es_problem.num_spatial_orbitals, es_problem.num_particles, converter)

        u = UCC(qubit_converter = converter,  num_spatial_orbitals = es_problem.num_spin_orbitals//2, 
                num_particles = (es_problem.num_alpha, es_problem.num_beta), excitations = 'sd')
        u = u.decompose(reps = 2)
        if ret == "circ":
            return u,qc
        u = qc.compose(u)
#         u = u.decompose(reps = 2)
#         for inst in u:
    #         "fountain"
    #         print(inst)
    #         inst.synthesic = LieTrotter("fountain")
#             qc.append(inst)
        print( u.decompose(reps = 0) )
        h = generate_preset_pass_manager(optimization_level = 1)
        print(h.run(u))
        b = u.decompose(reps = 3).num_nonlocal_gates()
        u = pass_manager.run(u)
        print( u.decompose(reps = 2) )
        u = u
        if u.decompose(reps = 3).num_nonlocal_gates() < k:
            k = u.decompose(reps = 3).num_nonlocal_gates()
            b1 = b 
            trans1 = trans
#     u.draw(output = "latex", scale = 0.7)
    print("2-qubits gates " + name + " = ", b1)
    print("2-qubits gates " + name + "_opt = ", k)
    print(trans1)

    return k,b1

gate_number("BK")
#

          ┌─────────────────────┐┌─────────────────────┐┌─────────────────────┐»
q_0: ─────┤0                    ├┤0                    ├┤0                    ├»
     ┌───┐│                     ││                     ││                     │»
q_1: ┤ X ├┤1                    ├┤1                    ├┤1                    ├»
     └───┘│  exp(it IYII)(t[0]) ││  exp(it ZYZI)(t[0]) ││  exp(it IIIY)(t[1]) │»
q_2: ─────┤2                    ├┤2                    ├┤2                    ├»
          │                     ││                     ││                     │»
q_3: ─────┤3                    ├┤3                    ├┤3                    ├»
          └─────────────────────┘└─────────────────────┘└─────────────────────┘»
«     ┌─────────────────────┐┌─────────────────────┐┌─────────────────────┐»
«q_0: ┤0                    ├┤0                    ├┤0                    ├»
«     │                     ││                     ││                     │»
«q_1: ┤1                    ├┤1         

(22, 38)

In [26]:
u,qc = gate_number("BK",[0,1,2,3],ret = "circ")

In [66]:
ins = [inst for inst in u]
trans = [0,6,1,4,2,8,3,10,5,11,7,9]
trans = [0,2,1,11,5,10,4,3,9,6,7,8]
trans = [0,1,2,3,4,5,6,7,8,9,10,11]
insts = [ins[i] for i in trans]
for inst in insts:
    qc.append(inst)
#     qc.barrier()
print(qc.decompose(reps = 1))
print(qc.decompose(reps = 3).num_nonlocal_gates())
# print(qc.decompose(reps = 3))
pass_manager = generate_preset_pass_manager(optimization_level = 3)

u = pass_manager.run(qc)
# u = pass_manager.run(u)
print(u.decompose(reps = 3))
print(u.decompose(reps = 3).num_nonlocal_gates())
# u = pass_manager.run(u)
# print(u.decompose(reps = 3))
# print(u.decompose(reps = 3).num_nonlocal_gates())
# insts = [ins[0],ins[6],ins[3],ins[10],ins[2],ins[8],ins[],ins[],ins[],ins[],]

     ┌───────────────┐┌─────┐┌───┐                                    ┌───┐»
q_0: ┤ Ry(-1.0*t[1]) ├┤ Sdg ├┤ H ├────────────────────────────────────┤ X ├»
     └─┬───────────┬─┘└─────┘└───┘┌───┐┌───┐┌──────────────┐┌───┐┌───┐└─┬─┘»
q_1: ──┤ U3(π,0,π) ├──────────────┤ X ├┤ X ├┤ Rz(1.0*t[0]) ├┤ X ├┤ X ├──■──»
     ┌─┴───────────┴─┐┌─────┐┌───┐└─┬─┘└─┬─┘└──────────────┘└─┬─┘└─┬─┘┌───┐»
q_2: ┤ Ry(-1.0*t[0]) ├┤ Sdg ├┤ H ├──■────┼────────────────────┼────■──┤ H ├»
     └───────────────┘└─────┘└───┘       │                    │       └───┘»
q_3: ────────────────────────────────────■────────────────────■────────────»
                                                                           »
«     ┌──────────────┐ ┌───┐ ┌───┐┌───┐┌───┐┌───┐┌───────────────┐┌───┐┌───┐»
«q_0: ┤ Rz(1.0*t[1]) ├─┤ X ├─┤ H ├┤ S ├┤ H ├┤ X ├┤ Rz(0.25*t[2]) ├┤ X ├┤ H ├»
«     └──────────────┘ └─┬─┘ └───┘└───┘└───┘└─┬─┘└───────────────┘└─┬─┘└───┘»
«q_1: ───────────────────■────────────────────┼─────────────────────┼────

In [6]:
from copy import copy
def compare(pauli1,pauli2,pos_tar):
    count = 0
    flag = False
    possible_target = []
    target = -1
    for i,gate in enumerate(pauli1):
        if not(gate == "I") and not(pauli2[i] == "I"):
            if (gate == pauli2[i]) and not(gate == "I"):
                count += 2
                if target < 0:
                    target = i
                    count -= 2
            elif gate + pauli2[i] in ["XY","YX"]:
                target = i
            else:
                possible_target.append(i)
                
#     for i,gate1 in enumerate(pauli1):
#         for j,gate2 in enumerate(pauli1[i+1:],i+1):
#             gate = gate1 + gate2
#             if gate in ["XY","YX"]:
#                 if gate == pauli2[j] + pauli2[i]:
#                     count += 1 
            
#     print("min = ", min(_count_nonI(pauli1), _count_nonI(pauli2)))               
#     intersection = list(set(possible_target) & set(b))
    if flag:
        return count
    if target == -1:
        print("-1")
    return count, target

def pauli_composing(pl):
#     создаю копию pl
    pauli_list = [pauli for pauli in pl]
    pauli_reorder = []
    targ = [-1]
    pauli_reorder.append(pauli_list[0])
    pauli_list.pop(0)
    r = 0
    while len(pauli_list) > 0:
        index = 0
        count = 0
        
        for i, pauli in enumerate(pauli_list):
            count0, target0 = com(pauli_reorder[-1],pauli)
            if  count0 > count:
                count = count0
                index = i
                target = target0
        r+=count
        pauli_reorder.append(pauli_list[index])
        i = 1
        targ.append(target)
        if target >= 0: 
            while targ[-1 - i] < 0 and i < len(targ) :
                targ[-1 - i] = target
                i += 1
        pauli_list.pop(index)
    return pauli_reorder, r
def count_length(pauli_list):
    l = 0 
    for pauli in pauli_list:
        l += len(pauli)
        for gate in pauli:
            if gate == "I":
                l-=1
        l -= 1
    return l*2
def target_detect(pauli_list):
    return 
u,qc = gate_number("BK",[0,1,2,3],ret = "circ")
ins = [inst for inst in u]
pauli_list = [inst.operation for inst in ins]
print(pauli_list)
p,r = pauli_composing(pauli_list)
print(pauli_list)
print(p)
print(r)
print("non_optimized = ", count_length(p))
print("optimized =", count_length(p) - r)

[Instruction(name='exp(it IYII)', num_qubits=4, num_clbits=0, params=[ParameterVectorElement(t[0])]), Instruction(name='exp(it ZYZI)', num_qubits=4, num_clbits=0, params=[ParameterVectorElement(t[0])]), Instruction(name='exp(it IIIY)', num_qubits=4, num_clbits=0, params=[ParameterVectorElement(t[1])]), Instruction(name='exp(it IIZY)', num_qubits=4, num_clbits=0, params=[ParameterVectorElement(t[1])]), Instruction(name='exp(it IYIX)', num_qubits=4, num_clbits=0, params=[ParameterVectorElement(t[2])]), Instruction(name='exp(it IXIY)', num_qubits=4, num_clbits=0, params=[ParameterVectorElement(t[2])]), Instruction(name='exp(it IYZX)', num_qubits=4, num_clbits=0, params=[ParameterVectorElement(t[2])]), Instruction(name='exp(it IXZY)', num_qubits=4, num_clbits=0, params=[ParameterVectorElement(t[2])]), Instruction(name='exp(it ZXZY)', num_qubits=4, num_clbits=0, params=[ParameterVectorElement(t[2])]), Instruction(name='exp(it ZYZX)', num_qubits=4, num_clbits=0, params=[ParameterVectorElemen

TypeError: 'Gate' object is not iterable

In [5]:
u,qc = gate_number("BK",[0,1,2,3],ret = "circ")
ins = [inst for inst in u]
pauli_list = [inst.operation.name[7:-1] for inst in ins]
print(pauli_list)
p,r = pauli_composing(pauli_list)
print(pauli_list)
print(p)
print(r)
print("non_optimized = ", count_length(p))
print("optimized =", count_length(p) - r)
ins2 = []
for i in range(len(ins)):
    ins2.append(ins[pauli_list.index(p[i])])
# ins2 = [ins[index] for inst in u]
ins = ins2
# for i in range(len(ins)):
#     ins[i].operation.name = inst.operation.name[0:7] + p[i] + inst.operation.name[-1] 
#     print(ins[i].operation.definition)
for inst in ins:
    qc.append(inst)
print(qc)
print(qc.decompose(reps = 1))
print(qc.decompose(reps = 3).num_nonlocal_gates())
# print(qc.dec)
# print(qc.decompose(reps = 3))
pass_manager = generate_preset_pass_manager(optimization_level = 3)

u = pass_manager.run(qc)
# u = pass_manager.run(u)
print(u.decompose(reps = 3))
print(u.decompose(reps = 3).num_nonlocal_gates())

['IYII', 'ZYZI', 'IIIY', 'IIZY', 'IYIX', 'IXIY', 'IYZX', 'IXZY', 'ZXZY', 'ZYZX', 'ZXIY', 'ZYIX']
['IYII', 'ZYZI', 'IIIY', 'IIZY', 'IYIX', 'IXIY', 'IYZX', 'IXZY', 'ZXZY', 'ZYZX', 'ZXIY', 'ZYIX']
['IYII', 'ZYZI', 'ZXZY', 'ZYZX', 'IYZX', 'IXZY', 'IIZY', 'IIIY', 'IYIX', 'ZYIX', 'ZXIY', 'IXIY']
25
non_optimized =  38
optimized = 13
          ┌─────────────────────┐┌─────────────────────┐┌─────────────────────┐»
q_0: ─────┤0                    ├┤0                    ├┤0                    ├»
     ┌───┐│                     ││                     ││                     │»
q_1: ┤ X ├┤1                    ├┤1                    ├┤1                    ├»
     └───┘│  exp(it IYII)(t[0]) ││  exp(it ZYZI)(t[0]) ││  exp(it ZXZY)(t[2]) │»
q_2: ─────┤2                    ├┤2                    ├┤2                    ├»
          │                     ││                     ││                     │»
q_3: ─────┤3                    ├┤3                    ├┤3                    ├»
          └────────────

In [3]:

from Ternary_Tree import evolve_pauli
qc1 = evolve_pauli(
    pauli = "YX",
    time = 1.0,
    cx_structure = "fountain",
    label = None,
    target = 0
)
qc2 = evolve_pauli(
    pauli = "XY",
    time = 1.0,
    cx_structure = "fountain",
    label = None,
    target = 0
)
qc = qc1.compose(qc2)
# qc.draw(output = "latex_source", scale = 0.7)
qc.draw(output = "latex_source", scale = 0.7)

YX
XY


'\\documentclass[border=2px]{standalone}\n\n\\usepackage[braket, qm]{qcircuit}\n\\usepackage{graphicx}\n\n\\begin{document}\n\\scalebox{0.7}{\n\\Qcircuit @C=1.0em @R=0.2em @!R { \\\\\n\t \t\\nghost{{q}_{0} :  } & \\lstick{{q}_{0} :  } & \\gate{\\mathrm{H}} & \\qw & \\targ & \\gate{\\mathrm{R_Z}\\,(\\mathrm{2})} & \\targ & \\gate{\\mathrm{H}} & \\gate{\\mathrm{S^\\dagger}} & \\gate{\\mathrm{H}} & \\targ & \\gate{\\mathrm{R_Z}\\,(\\mathrm{2})} & \\targ & \\gate{\\mathrm{H}} & \\gate{\\mathrm{S}} & \\qw & \\qw\\\\\n\t \t\\nghost{{q}_{1} :  } & \\lstick{{q}_{1} :  } & \\gate{\\mathrm{S^\\dagger}} & \\gate{\\mathrm{H}} & \\ctrl{-1} & \\qw & \\ctrl{-1} & \\gate{\\mathrm{H}} & \\gate{\\mathrm{S}} & \\gate{\\mathrm{H}} & \\ctrl{-1} & \\qw & \\ctrl{-1} & \\gate{\\mathrm{H}} & \\qw & \\qw & \\qw\\\\\n\\\\ }}\n\\end{document}'

In [14]:
from qiskit import QuantumCircuit,execute
import numpy as np
from qiskit import Aer

backend = Aer.get_backend('unitary_simulator')
qc = QuantumCircuit(1)
qc.u(np.pi/2,np.pi/4,0,0)
print(qc)
job = execute(qc, backend)
result = job.result()

print(result.get_unitary(qc, decimals=3))
# g = Gate.U(np.pi/2,np.pi/4,0)

   ┌──────────────┐
q: ┤ U(π/2,π/4,0) ├
   └──────────────┘
Operator([[ 0.707+0.j , -0.707+0.j ],
          [ 0.5  +0.5j,  0.5  +0.5j]],
         input_dims=(2,), output_dims=(2,))


In [1]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
import itertools
at_name = 'Li'
bond_length = 1
basis = 'sto-3g'
nmodes = 4
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
    basis=basis,
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
transformer = ActiveSpaceTransformer(num_electrons = es_problem.num_alpha + es_problem.num_beta - 2, 
                                     num_spatial_orbitals = nmodes//2  , active_orbitals = [1,2])
es_problem = transformer.transform(es_problem)
import numpy as np
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.synthesis import LieTrotter
from Ternary_Tree import BaseTree,TernaryTree, NodeInfo, EnumInfo, energy_classic, TernaryTreeMapper,TT_initial_state, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC
from qiskit import QuantumCircuit
pass_manager = generate_preset_pass_manager(optimization_level = 3)
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,EnumInfo(4), False])
parent_child[2] = NodeInfo(0,[4,EnumInfo(8), False])
parent_child[3] =  NodeInfo(1,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[4] = NodeInfo(2,[EnumInfo(5),EnumInfo(6),EnumInfo(7)])
# tt = BaseTree(4)
tt = TernaryTree(5, parent_child = parent_child)
print(tt)
print(tt.parent_child)
# tt = BaseTree
trans = [0,1,2,3]
# mapper = TernaryTreeMapper(es_problem,trans, tt = tt,enum_list = [[0,4],[1,5],[2,6],[3,7]])
# mapper = TernaryTreeMapper(es_problem,trans, tt = tt)
mapper = TernaryTreeMapper(es_problem,trans, tt = tt,enum_list = [[0,1],[2,3],[4,5],[6,7]])

converter = QubitConverter(mapper)
# print(converter)
# qc = TT_initial_state(es_problem.num_spin_orbitals, es_problem.num_particles, tt = tt)
u = UCC(qubit_converter = converter,  num_spatial_orbitals = es_problem.num_spin_orbitals//2, 
    num_particles = (es_problem.num_alpha, es_problem.num_beta), excitations = 'sd')
u = u.decompose(reps = 2)
print(u)
# u = qc.compose(u)

4
0X  0X  0X  0X  0Y  0Y  0Y  0Y  
1X  1X  1X  1Y  2X  2X  2X  2Y  
3X  3Y  3Z      4X  4Y  4Z      
1   2   3   4   5   6   7   8   
 ---------------------------------------------------------------
{0: -1,[1, 2, False], 1: 0,[3, '4', False], 2: 0,[4, '8', False], 3: 1,['1', '2', '3'], 4: 2,['5', '6', '7']}
     ┌──────────────────────┐┌──────────────────────┐┌──────────────────────┐»
q_0: ┤0                     ├┤0                     ├┤0                     ├»
     │                      ││                      ││                      │»
q_1: ┤1                     ├┤1                     ├┤1                     ├»
     │                      ││                      ││                      │»
q_2: ┤2 exp(it IIIYI)(t[0]) ├┤2 exp(it IZIYI)(t[0]) ├┤2 exp(it IIIIY)(t[1]) ├»
     │                      ││                      ││                      │»
q_3: ┤3                     ├┤3                     ├┤3                     ├»
     │                      ││                      ││     

In [12]:
from Ternary_Tree import evolve_pauli, pauli_composing
from qiskit import QuantumCircuit

ins = [inst for inst in u]
qc = QuantumCircuit(5)
print(ins[1].operation.params[0])
print(type(ins[1].operation.params[0]))
# for inst in ins:
#     qc.append(inst)

# print(qc.decompose(reps = 3).num_nonlocal_gates())
# qc  = qc.decompose(reps = 3)

# pass_manager = generate_preset_pass_manager(optimization_level = 3)

# pauli_list = [inst.operation.name[7:-1] for inst in ins]
# print(pauli_list)
# p,targ, r = pauli_composing(pauli_list)
# print(p)
# print(targ)
# print("r = ", r)
# u0 = QuantumCircuit(5)
# for i,pauli in enumerate(p):
#     qc = evolve_pauli(
#         pauli = pauli,
#         time = i,
#         cx_structure = "fountain",
#         label = None,
#         target = targ[i]
#     )

#     u0 = u0.compose(qc)
# print(u0)
# u0  = u0.decompose(reps = 4)
# u0 = pass_manager.run(u0)
# print(u0.decompose(reps = 4))
# u0 = pass_manager.run(u0)
# print(u0.num_nonlocal_gates())



t[0]
<class 'qiskit.circuit.parametervector.ParameterVectorElement'>


In [1]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC

from Ternary_Tree import *
import itertools

at_name = 'Li'
bond_length = 1
basis = 'sto-3g'
nmodes = 4
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
    basis=basis,
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
transformer = ActiveSpaceTransformer(num_electrons = es_problem.num_alpha + es_problem.num_beta - 2, 
                                     num_spatial_orbitals = nmodes//2  , active_orbitals = [1,2])
es_problem = transformer.transform(es_problem)



parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,EnumInfo(4), False])
parent_child[2] = NodeInfo(0,[4, EnumInfo(8), False])
parent_child[3] =  NodeInfo(1,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[4] = NodeInfo(2,[EnumInfo(5),EnumInfo(6),EnumInfo(7)])

# tt = BaseTree(4)
tt = TernaryTree(parent_child = parent_child)
print(tt)
print(tt.parent_child)
# tt = BaseTree
trans = [0,1,2,3,4,5,6,7]
# mapper = TernaryTreeMapper(es_problem,trans, tt = tt,enum_list = [[0,4],[1,5],[2,6],[3,7]])
mapper = TernaryTreeMapper(es_problem, tt = tt)
# mapper = TernaryTreeMapper(es_problem,trans)
# mapper = TernaryTreeMapper(es_problem, trans, enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11],[12,13],[14,15]])
# mapper = JordanWignerMapper(trans)
# mapper = BravyiKitaevMapper(trans)
converter = QubitConverter(mapper)
# print(converter)
qc = TT_initial_state(es_problem.num_spin_orbitals, es_problem.num_particles, tt = tt)
u = UCC(qubit_converter = converter,  num_spatial_orbitals = es_problem.num_spin_orbitals//2, 
    num_particles = (es_problem.num_alpha, es_problem.num_beta), excitations = 'sd')
u = u.decompose(reps = 2)
# print(u)

u0, _ =optimize_ucc(u) 
print(u0.num_nonlocal_gates())


4
0X  0X  0X  0X  0Y  0Y  0Y  0Y  
1X  1X  1X  1Y  2X  2X  2X  2Y  
3X  3Y  3Z      4X  4Y  4Z      
1   2   3   4   5   6   7   8   
 ---------------------------------------------------------------
{0: -1,[1, 2, False], 1: 0,[3, '4', False], 2: 0,[4, '8', False], 3: 1,['1', '2', '3'], 4: 2,['5', '6', '7']}
asdf
0X  0X  0X  0X  0Y  0Y  0Y  0Y  
1X  1X  1X  1Y  2X  2X  2X  2Y  
3X  3Y  3Z      4X  4Y  4Z      
1   3   4   2   6   8   7   5   
 ---------------------------------------------------------------
['IZIYI', 'IIZIY']
[[('XXIXI', 'XYIII'), ('XXIYI', 'XXIZI'), ('YIYII', 'YIXIX'), ('YIXIZ', 'YIXIY')], [[1, (-0-1j)], [1, (-0-1j)], [1, (-0-1j)], [1, (-0-1j)]]]
global phase: -2.0*t[0] - 2.0*t[1] - 8.0*t[2] + 18π
      ┌──────────────┐                                                      »
q_0: ─┤ U(π/2,0,π/2) ├────────────────────────■─────────────────────────────»
     ┌┴──────────────┤┌───┐┌───────────────┐┌─┴─┐┌───┐┌───────────────┐┌───┐»
q_1: ┤ U(0,0,2*t[0]) ├┤ X ├┤ U(0,0,2*t[0]) 

In [1]:
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4, False])
parent_child[2] = NodeInfo(0,[5,6, False])
parent_child[3] =  NodeInfo(1,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[4] = NodeInfo(2,[EnumInfo(4),EnumInfo(5),EnumInfo(6)])
parent_child[5] =  NodeInfo(1,[EnumInfo(7),EnumInfo(8),EnumInfo(9)])
parent_child[6] =  NodeInfo(1,[EnumInfo(10),EnumInfo(11),EnumInfo(12)])

NameError: name 'NodeInfo' is not defined

In [8]:
from qiskit import QuantumCircuit
u = QuantumCircuit(6)
print(u.num_qubits)

6


# В этой ячейке содежатся parent_child для тестируемых деревье

In [ ]:
nmodes = 4

#binary_tree
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,EnumInfo(4), False])
parent_child[2] = NodeInfo(0,[5, EnumInfo(8), False])
parent_child[3] =  NodeInfo(1,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[4] = NodeInfo(2,[EnumInfo(5),EnumInfo(6),EnumInfo(7)])



In [ ]:
nmodes = 6

#binary_tree
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4, False])
parent_child[2] = NodeInfo(0,[5,6, False])
parent_child[3] =  NodeInfo(1,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[4] = NodeInfo(2,[EnumInfo(4),EnumInfo(5),EnumInfo(6)])
parent_child[5] =  NodeInfo(1,[EnumInfo(7),EnumInfo(8),EnumInfo(9)])
parent_child[6] =  NodeInfo(1,[EnumInfo(10),EnumInfo(11),EnumInfo(12)])

#bin_alpha_beta_tree
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4, False])
parent_child[2] = NodeInfo(0,[5,6, False])
parent_child[3] =  NodeInfo(1,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[4] = NodeInfo(2,[EnumInfo(4),EnumInfo(5),EnumInfo(6)])
parent_child[5] =  NodeInfo(1,[EnumInfo(7),EnumInfo(8),EnumInfo(9)])
parent_child[6] =  NodeInfo(1,[EnumInfo(10),EnumInfo(11),EnumInfo(12)])

In [ ]:
nmodes = 8

# binary tree
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4, False])
parent_child[2] = NodeInfo(0,[5,6, False])
parent_child[3] = NodeInfo(1,[7,8, False])
parent_child[4] = NodeInfo(1,[EnumInfo(7),EnumInfo(8), False])
parent_child[5] = NodeInfo(2,[9,10, False])
parent_child[6] = NodeInfo(2,[EnumInfo(15),EnumInfo(16), False])
parent_child[7] =  NodeInfo(3,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[8] = NodeInfo(3,[EnumInfo(4),EnumInfo(5),EnumInfo(6)])
parent_child[9] =  NodeInfo(5,[EnumInfo(9),EnumInfo(10),EnumInfo(11)])
parent_child[10] =  NodeInfo(5,[EnumInfo(12),EnumInfo(13),EnumInfo(14)])


#bin_alpha_beta
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4,5])
parent_child[2] = NodeInfo(0,[6,7, 8])
parent_child[3] = NodeInfo(1,[EnumInfo(1),EnumInfo(2), EnumInfo(3)])
parent_child[4] = NodeInfo(1,[EnumInfo(4),EnumInfo(5), EnumInfo(6)])
parent_child[5] = NodeInfo(1,[EnumInfo(7),EnumInfo(8), False])
parent_child[6] = NodeInfo(2,[EnumInfo(9),EnumInfo(10), EnumInfo(11)])
parent_child[7] = NodeInfo(2,[EnumInfo(12),EnumInfo(13),EnumInfo(14)])
parent_child[8] = NodeInfo(2,[EnumInfo(15),EnumInfo(16),False])

In [ ]:
nmodes = 10 

# bin_alpha_beta
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4,5])
parent_child[2] = NodeInfo(0,[6,7, 8])
parent_child[3] = NodeInfo(1,[9 ,EnumInfo(4), EnumInfo(5)])
parent_child[4] = NodeInfo(1,[EnumInfo(6),EnumInfo(6), EnumInfo(8)])
parent_child[5] = NodeInfo(1,[EnumInfo(9),EnumInfo(10), False])
parent_child[6] = NodeInfo(2,[10,EnumInfo(14), EnumInfo(15)])
parent_child[7] = NodeInfo(2,[EnumInfo(16),EnumInfo(17),EnumInfo(18)])
parent_child[8] = NodeInfo(2,[EnumInfo(19),EnumInfo(20),False])
parent_child[9] = NodeInfo(3,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[10] = NodeInfo(6,[EnumInfo(11),EnumInfo(12),EnumInfo(13)])

In [ ]:
nmodes = 12 
# bin_alpha_beta
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4,5])
parent_child[2] = NodeInfo(0,[6,7, 8])
parent_child[3] = NodeInfo(1,[9 ,10, EnumInfo(7)])
parent_child[4] = NodeInfo(1,[EnumInfo(8),EnumInfo(9), EnumInfo(10)])
parent_child[5] = NodeInfo(1,[EnumInfo(11),EnumInfo(12), False])
parent_child[6] = NodeInfo(2,[11,12, EnumInfo(19)])
parent_child[7] = NodeInfo(2,[EnumInfo(20),EnumInfo(21),EnumInfo(22)])
parent_child[8] = NodeInfo(2,[EnumInfo(23),EnumInfo(24),False])
parent_child[9] = NodeInfo(3,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[10] = NodeInfo(3,[EnumInfo(4),EnumInfo(5),EnumInfo(6)])
parent_child[11] = NodeInfo(6,[EnumInfo(13),EnumInfo(14),EnumInfo(15)])
parent_child[12] = NodeInfo(6,[EnumInfo(16),EnumInfo(17),EnumInfo(18)])

In [ ]:
nmodes = 14
# bin_alpha_beta
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4,5])
parent_child[2] = NodeInfo(0,[6,7, 8])
parent_child[3] = NodeInfo(1,[9 ,10, 11])
parent_child[4] = NodeInfo(1,[EnumInfo(10),EnumInfo(11), EnumInfo(12)])
parent_child[5] = NodeInfo(1,[EnumInfo(13),EnumInfo(14), False])
parent_child[6] = NodeInfo(2,[12,13, 14])
parent_child[7] = NodeInfo(2,[EnumInfo(24),EnumInfo(25),EnumInfo(26)])
parent_child[8] = NodeInfo(2,[EnumInfo(27),EnumInfo(28),False])

parent_child[9] = NodeInfo(3,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[10] = NodeInfo(3,[EnumInfo(4),EnumInfo(5),EnumInfo(6)])
parent_child[11] = NodeInfo(3,[EnumInfo(7),EnumInfo(8),EnumInfo(9)])

parent_child[12] = NodeInfo(6,[EnumInfo(15),EnumInfo(16),EnumInfo(17)])
parent_child[13] = NodeInfo(6,[EnumInfo(18),EnumInfo(19),EnumInfo(20)])

parent_child[14] = NodeInfo(6,[EnumInfo(21),EnumInfo(22),EnumInfo(23)])

# Далее выводятся результаты для различных кодировок с использованием моей схемы оптимизации

In [1]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC

from Ternary_Tree import *
import itertools

at_name = 'Li'
bond_length = 1
basis = 'sto-3g'

def print_data(mapper,name):
    converter = QubitConverter(mapper)
    u = UCC(qubit_converter = converter,  num_spatial_orbitals = es_problem.num_spin_orbitals//2, 
        num_particles = (es_problem.num_alpha, es_problem.num_beta), excitations = 'sd')
    u = u.decompose(reps = 2)
#     print(u)
    print(name)
    print("non-optimized = ",u.decompose(reps = 4).num_nonlocal_gates())
    u0, _ =optimize_ucc(u) 
    print("optimized = ",u0.num_nonlocal_gates())

## nmodes = 4, HLi

In [2]:
nmodes = 4
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
    basis=basis,
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
transformer = ActiveSpaceTransformer(num_electrons = es_problem.num_alpha + es_problem.num_beta - 2, 
                                     num_spatial_orbitals = nmodes//2  , active_orbitals = [1,2])
es_problem = transformer.transform(es_problem)
# ----------------------------------------------
mapper = JordanWignerMapper()
print_data(mapper,"JW")
mapper = BravyiKitaevMapper()
print_data(mapper,"BK")
mapper = TernaryTreeMapper(es_problem)
print_data(mapper,"TT, st_enum")
mapper = TernaryTreeMapper(es_problem,enum_list = [[0,1],[2,3],[4,5],[6,7]])
print_data(mapper,"TT, order_enum")

parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,EnumInfo(4), False])
parent_child[2] = NodeInfo(0,[4, EnumInfo(8), False])
parent_child[3] =  NodeInfo(1,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[4] = NodeInfo(2,[EnumInfo(5),EnumInfo(6),EnumInfo(7)])

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt)
print_data(mapper,"Bin_tt")

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt, enum_list = [[0,1],[2,3],[4,5],[6,7]])
print_data(mapper,"Bin_tt, order_num")

JW
non-optimized =  56
global phase: -2.0*t[0] - 2.0*t[1] - 8.0*t[2] + 142.157067574938
     ┌──────────────┐     ┌──────────────┐                      »
q_0: ┤ U(π/2,0,π/2) ├──■──┤ U(π/2,π/2,0) ├──────────────────────»
     └┬────────────┬┘  │  └──────────────┘                      »
q_1: ─┤ U(π/2,0,π) ├───┼─────────■───────────────────────────■──»
     ┌┴────────────┴┐  │         │                           │  »
q_2: ┤ U(π/2,0,π/2) ├──┼─────────┼───────────────────────────┼──»
     └──────────────┘┌─┴─┐     ┌─┴─┐      ┌───────────────┐┌─┴─┐»
q_3: ────────────────┤ X ├─────┤ X ├──────┤ U(0,0,2*t[0]) ├┤ X ├»
                     └───┘     └───┘      └───────────────┘└───┘»
«                                ┌─────────────┐                             »
«q_0: ──────────────────────■────┤ U(π/2,-π,0) ├─────────────────────────────»
«     ┌─────────────────┐   │    └─────────────┘                             »
«q_1: ┤ U(π/2,π/2,-π/2) ├───┼───────────────────────■─────────────────────■──»
« 

global phase: -2.0*t[0] - 1.0*t[1] - 8.0*t[2] + 87.8936457732121
                      ┌───┐                 ┌───┐┌───┐┌───────────────┐┌───┐»
q_0: ─────────────────┤ X ├─────────────────┤ X ├┤ X ├┤ U(0,0,2*t[0]) ├┤ X ├»
       ┌────────────┐ └─┬─┘┌───────────────┐└─┬─┘└─┬─┘└───────────────┘└─┬─┘»
q_1: ──┤ U(π/2,0,π) ├───■──┤ U(π/2,-π,π/4) ├──┼────┼─────────────────────┼──»
     ┌─┴────────────┴┐     └───────────────┘  │    │                     │  »
q_2: ┤ U(0,0,2*t[0]) ├────────────────────────■────┼─────────────────────┼──»
     └───────────────┘                             │                     │  »
q_3: ──────────────────────────────────────────────■─────────────────────■──»
                                                                            »
«     ┌───┐┌─────────────────┐     ┌────────────────┐┌───┐┌───┐»
«q_0: ┤ X ├┤ U(π/2,-π/2,π/2) ├──■──┤ U(0,-π/4,-π/4) ├┤ X ├┤ X ├»
«     └─┬─┘└─────────────────┘┌─┴─┐├────────────────┤└─┬─┘└─┬─┘»
«q_1: ──┼─────────────────────┤ X ├┤ U

global phase: -1.0*t[0] - 1.0*t[1] - 8.0*t[2] + 38π
                           ┌───┐      ┌───┐┌───┐┌───────────────┐»
q_0: ──────────────────────┤ X ├──────┤ X ├┤ X ├┤ U(0,0,2*t[0]) ├»
       ┌────────────┐      └─┬─┘      └─┬─┘└─┬─┘└───────────────┘»
q_1: ──┤ U(π/2,0,π) ├────────■──────────┼────┼───────────────────»
     ┌─┴────────────┴┐┌──────────────┐  │    │                   »
q_2: ┤ U(2*t[0],0,0) ├┤ U(π/2,0,π/2) ├──■────┼───────────────────»
     └───────────────┘└──────────────┘       │                   »
q_3: ────────────────────────────────────────■───────────────────»
                                                                 »
«     ┌───────────────┐┌───┐     ┌───┐      ┌───┐                             »
«q_0: ┤ U(0,0,2*t[2]) ├┤ X ├─────┤ X ├──────┤ X ├──■──────────────────────────»
«     └───────────────┘└─┬─┘     └─┬─┘      └─┬─┘  │  ┌─────────────────┐     »
«q_1: ───────────────────┼─────────┼──────────■────┼──┤ U(π/2,π/2,-π/2) ├──■──»
«                        

## nmodes = 6

In [3]:
nmodes = 6
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
    basis=basis,
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
transformer = ActiveSpaceTransformer(num_electrons = es_problem.num_alpha + es_problem.num_beta - 2, 
                                     num_spatial_orbitals = nmodes//2  , active_orbitals = [1,2,3])
es_problem = transformer.transform(es_problem)
# ----------------------------------------------
mapper = JordanWignerMapper()
print_data(mapper,"JW")
mapper = BravyiKitaevMapper()
print_data(mapper,"BK")
mapper = TernaryTreeMapper(es_problem)
print_data(mapper,"TT, st_enum")
mapper = TernaryTreeMapper(es_problem,enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11]])
print_data(mapper,"TT, order_enum")

parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4, False])
parent_child[2] = NodeInfo(0,[5,6, False])
parent_child[3] =  NodeInfo(1,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[4] = NodeInfo(2,[EnumInfo(4),EnumInfo(5),EnumInfo(6)])
parent_child[5] =  NodeInfo(1,[EnumInfo(7),EnumInfo(8),EnumInfo(9)])
parent_child[6] =  NodeInfo(1,[EnumInfo(10),EnumInfo(11),EnumInfo(12)])

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt)
print_data(mapper,"Bin_tt")

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt, enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11]])
print_data(mapper,"Bin_tt, order_num")

JW
non-optimized =  280
optimized =  128
BK
non-optimized =  304
optimized =  155
TT, st_enum
non-optimized =  240
optimized =  104
TT, order_enum
non-optimized =  264
optimized =  132
6
Bin_tt
non-optimized =  192
optimized =  82
6
Bin_tt, order_num
non-optimized =  280
optimized =  93


## nmodes = 6 {0,1,2}

In [4]:
nmodes = 6
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
    basis=basis,
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
transformer = ActiveSpaceTransformer(num_electrons = es_problem.num_alpha + es_problem.num_beta, 
                                     num_spatial_orbitals = nmodes//2  , active_orbitals = [0,1,2])
es_problem = transformer.transform(es_problem)
# ----------------------------------------------
mapper = JordanWignerMapper()
print_data(mapper,"JW")
mapper = BravyiKitaevMapper()
print_data(mapper,"BK")
mapper = TernaryTreeMapper(es_problem)
print_data(mapper,"TT, st_enum")
mapper = TernaryTreeMapper(es_problem,enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11]])
print_data(mapper,"TT, order_enum")

parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4, False])
parent_child[2] = NodeInfo(0,[5,6, False])
parent_child[3] =  NodeInfo(1,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[4] = NodeInfo(1,[EnumInfo(4),EnumInfo(5),EnumInfo(6)])
parent_child[5] =  NodeInfo(2,[EnumInfo(7),EnumInfo(8),EnumInfo(9)])
parent_child[6] =  NodeInfo(2,[EnumInfo(10),EnumInfo(11),EnumInfo(12)])

tt = TernaryTree(parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt)
print_data(mapper,"Bin_tt")

tt = TernaryTree(parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt, enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11]])
print_data(mapper,"Bin_tt, order_num")

JW
non-optimized =  280
optimized =  123
BK
non-optimized =  266
optimized =  100
TT, st_enum
non-optimized =  192
optimized =  122
TT, order_enum
non-optimized =  264
optimized =  114
6
Bin_tt
non-optimized =  192
optimized =  67
6
Bin_tt, order_num
non-optimized =  280
optimized =  144


## nmodes = 8

In [2]:
nmodes = 8
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
    basis=basis,
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
transformer = ActiveSpaceTransformer(num_electrons = es_problem.num_alpha + es_problem.num_beta, 
                                     num_spatial_orbitals = nmodes//2  , active_orbitals = [0,1,2,3])
es_problem = transformer.transform(es_problem)
# ----------------------------------------------
mapper = JordanWignerMapper()
print_data(mapper,"JW")
mapper = BravyiKitaevMapper()
print_data(mapper,"BK")
mapper = TernaryTreeMapper(es_problem)
print_data(mapper,"TT, st_enum")
mapper = TernaryTreeMapper(es_problem,enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11],[12,13],[14,15]])
print_data(mapper,"TT, order_enum")

# bin_alpha_beta
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4,5])
parent_child[2] = NodeInfo(0,[6,7, 8])
parent_child[3] = NodeInfo(1,[9 ,EnumInfo(4), EnumInfo(5)])
parent_child[4] = NodeInfo(1,[EnumInfo(6),EnumInfo(6), EnumInfo(8)])
parent_child[5] = NodeInfo(1,[EnumInfo(9),EnumInfo(10), False])
parent_child[6] = NodeInfo(2,[10,EnumInfo(14), EnumInfo(15)])
parent_child[7] = NodeInfo(2,[EnumInfo(16),EnumInfo(17),EnumInfo(18)])
parent_child[8] = NodeInfo(2,[EnumInfo(19),EnumInfo(20),False])
parent_child[9] = NodeInfo(3,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[10] = NodeInfo(6,[EnumInfo(11),EnumInfo(12),EnumInfo(13)])

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt)
print_data(mapper,"Bin_tt")

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt, enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11],[12,13],[14,15]])
print_data(mapper,"Bin_tt, order_num")

JW
non-optimized =  1440
optimized =  536
BK
non-optimized =  1316
optimized =  416
None
TT, st_enum
non-optimized =  1248
optimized =  448
[[0, 1], [2, 3], [4, 5], [6, 7], [8, 9], [10, 11], [12, 13], [14, 15]]
TT, order_enum
non-optimized =  1272
optimized =  502
10
None
Bin_tt
non-optimized =  1556
optimized =  382
10
[[0, 1], [2, 3], [4, 5], [6, 7], [8, 9], [10, 11], [12, 13], [14, 15]]
Bin_tt, order_num
non-optimized =  1556
optimized =  382


In [3]:
parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4,5])
parent_child[2] = NodeInfo(0,[6,7, 8])
parent_child[3] = NodeInfo(1,[EnumInfo(1),EnumInfo(2), EnumInfo(3)])
parent_child[4] = NodeInfo(1,[EnumInfo(4),EnumInfo(5), EnumInfo(6)])
parent_child[5] = NodeInfo(1,[EnumInfo(7),EnumInfo(8), False])
parent_child[6] = NodeInfo(2,[EnumInfo(9),EnumInfo(10), EnumInfo(11)])
parent_child[7] = NodeInfo(2,[EnumInfo(12),EnumInfo(13),EnumInfo(14)])
parent_child[8] = NodeInfo(2,[EnumInfo(15),EnumInfo(16),False])

tt = TernaryTree(parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt)
print_data(mapper,"Bin_tt")

8
None
Bin_tt
non-optimized =  1272
optimized =  370


## nmodes  =  10

In [14]:
nmodes = 10
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
    basis=basis,
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
transformer = ActiveSpaceTransformer(num_electrons = es_problem.num_alpha + es_problem.num_beta, 
                                     num_spatial_orbitals = nmodes//2  , active_orbitals = [0,1,2,3,4])
es_problem = transformer.transform(es_problem)
# ----------------------------------------------
mapper = JordanWignerMapper()
print_data(mapper,"JW")
mapper = BravyiKitaevMapper()
print_data(mapper,"BK")
mapper = TernaryTreeMapper(es_problem)
print_data(mapper,"TT, st_enum")
mapper = TernaryTreeMapper(es_problem,enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11],[12,13],[14,15],[16,17],[18,19]])
print_data(mapper,"TT, order_enum")

parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4,5])
parent_child[2] = NodeInfo(0,[6,7, 8])
parent_child[3] = NodeInfo(1,[9 ,EnumInfo(4), EnumInfo(5)])
parent_child[4] = NodeInfo(1,[EnumInfo(6),EnumInfo(6), EnumInfo(8)])
parent_child[5] = NodeInfo(1,[EnumInfo(9),EnumInfo(10), False])
parent_child[6] = NodeInfo(2,[10,EnumInfo(14), EnumInfo(15)])
parent_child[7] = NodeInfo(2,[EnumInfo(16),EnumInfo(17),EnumInfo(18)])
parent_child[8] = NodeInfo(2,[EnumInfo(19),EnumInfo(20),False])
parent_child[9] = NodeInfo(3,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[10] = NodeInfo(6,[EnumInfo(11),EnumInfo(12),EnumInfo(13)])

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt)
print_data(mapper,"Bin_tt")

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt, enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11],[12,13],[14,15],[16,17],[18,19]])
print_data(mapper,"Bin_tt, order_num")



JW
non-optimized =  3896
optimized =  1202
BK
non-optimized =  3952
optimized =  1068
TT, st_enum
non-optimized =  3092
optimized =  988
TT, order_enum
non-optimized =  3382
optimized =  1206
10
Bin_tt
non-optimized =  3248
optimized =  574
10
Bin_tt, order_num
non-optimized =  3748
optimized =  909


In [11]:
nmodes = 12
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
    basis=basis,
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
transformer = ActiveSpaceTransformer(num_electrons = es_problem.num_alpha + es_problem.num_beta, 
                                     num_spatial_orbitals = nmodes//2  , active_orbitals = [0,1,2,3,4,5])
es_problem = transformer.transform(es_problem)
# ----------------------------------------------
mapper = JordanWignerMapper()
print_data(mapper,"JW")
mapper = BravyiKitaevMapper()
print_data(mapper,"BK")
mapper = TernaryTreeMapper(es_problem)
print_data(mapper,"TT, st_enum")
mapper = TernaryTreeMapper(es_problem,enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11],[12,13],[14,15],[16,17],[18,19],[20,21],[22,23]])
print_data(mapper,"TT, order_enum")

parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4,5])
parent_child[2] = NodeInfo(0,[6,7, 8])
parent_child[3] = NodeInfo(1,[9 ,10, EnumInfo(7)])
parent_child[4] = NodeInfo(1,[EnumInfo(8),EnumInfo(9), EnumInfo(10)])
parent_child[5] = NodeInfo(1,[EnumInfo(11),EnumInfo(12), False])
parent_child[6] = NodeInfo(2,[11,12, EnumInfo(19)])
parent_child[7] = NodeInfo(2,[EnumInfo(20),EnumInfo(21),EnumInfo(22)])
parent_child[8] = NodeInfo(2,[EnumInfo(23),EnumInfo(24),False])
parent_child[9] = NodeInfo(3,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[10] = NodeInfo(3,[EnumInfo(4),EnumInfo(5),EnumInfo(6)])
parent_child[11] = NodeInfo(6,[EnumInfo(13),EnumInfo(14),EnumInfo(15)])
parent_child[12] = NodeInfo(6,[EnumInfo(16),EnumInfo(17),EnumInfo(18)])

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt)
print_data(mapper,"Bin_tt")

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt, enum_list = 
                           [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11],[12,13],[14,15],[16,17],[18,19],[20,21],[22,23]])
print_data(mapper,"Bin_tt, order_num")


JW
non-optimized =  8064
optimized =  2108
BK
non-optimized =  8680
optimized =  1748
TT, st_enum
non-optimized =  7040
optimized =  1140
TT, order_enum
non-optimized =  7536
optimized =  2114
12
Bin_tt
non-optimized =  6440
optimized =  1493
12
Bin_tt, order_num
non-optimized =  6712
optimized =  3145


In [13]:
nmodes = 14
basis = "6-31G"
driver = PySCFDriver(
    atom="H 0 0 0;" + at_name + " 0 0 " + str(bond_length),
#         basis="sto3g",
    basis=basis,
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()
transformer = ActiveSpaceTransformer(num_electrons = es_problem.num_alpha + es_problem.num_beta, 
                                     num_spatial_orbitals = nmodes//2  , active_orbitals = [0,1,2,3,4,5,6])
es_problem = transformer.transform(es_problem)
# ----------------------------------------------
mapper = JordanWignerMapper()
print_data(mapper,"JW")
mapper = BravyiKitaevMapper()
print_data(mapper,"BK")
mapper = TernaryTreeMapper(es_problem)
print_data(mapper,"TT, st_enum")
mapper = TernaryTreeMapper(es_problem,enum_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11],[12,13],[14,15],[16,17],[18,19],[20,21],[22,23],[24,25],[26,27]])
print_data(mapper,"TT, order_enum")

parent_child = {0 : NodeInfo(-1,[1,2,False])}
parent_child[1] = NodeInfo(0,[3,4,5])
parent_child[2] = NodeInfo(0,[6,7, 8])
parent_child[3] = NodeInfo(1,[9 ,10, 11])
parent_child[4] = NodeInfo(1,[EnumInfo(10),EnumInfo(11), EnumInfo(12)])
parent_child[5] = NodeInfo(1,[EnumInfo(13),EnumInfo(14), False])
parent_child[6] = NodeInfo(2,[12,13, 14])
parent_child[7] = NodeInfo(2,[EnumInfo(24),EnumInfo(25),EnumInfo(26)])
parent_child[8] = NodeInfo(2,[EnumInfo(27),EnumInfo(28),False])
parent_child[9] = NodeInfo(3,[EnumInfo(1),EnumInfo(2),EnumInfo(3)])
parent_child[10] = NodeInfo(3,[EnumInfo(4),EnumInfo(5),EnumInfo(6)])
parent_child[11] = NodeInfo(3,[EnumInfo(7),EnumInfo(8),EnumInfo(9)])
parent_child[12] = NodeInfo(6,[EnumInfo(15),EnumInfo(16),EnumInfo(17)])
parent_child[13] = NodeInfo(6,[EnumInfo(18),EnumInfo(19),EnumInfo(20)])
parent_child[14] = NodeInfo(6,[EnumInfo(21),EnumInfo(22),EnumInfo(23)])

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt)
print_data(mapper,"Bin_tt")

tt = TernaryTree(5, parent_child = parent_child)
mapper = TernaryTreeMapper(es_problem, tt = tt, enum_list = 
                           [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11],[12,13],[14,15],[16,17],[18,19],[20,21],[22,23],[24,25],[26,27]])
print_data(mapper,"Bin_tt, order_num")


JW
non-optimized =  14360
optimized =  3228
BK
non-optimized =  12396
optimized =  2913
TT, st_enum
non-optimized =  10716
optimized =  2189
TT, order_enum
non-optimized =  11698
optimized =  2438
14
Bin_tt
non-optimized =  10656
optimized =  2036
14
Bin_tt, order_num
non-optimized =  10640
optimized =  3706
